In [1]:
import pandas as pd

# Load data from Excel
sheet1_df = pd.read_excel('Mass Update.xlsx',skiprows=2)
# Rename the 'SKU' column to 'Reference_Code'
sheet1_df.rename(columns={'SKU': 'Reference_Code'}, inplace=True)
#print(sheet1_df)

sheet1_df = sheet1_df[['Reference_Code']]

#print(sheet1_df)

               Reference_Code
0                         NaN
1                         NaN
2                         NaN
3     H431-Hitambatik-Allsize
4     H431-Hitambatik-Bigsize
...                       ...
2249                      NaN
2250                      NaN
2251                      NaN
2252                      NaN
2253                      NaN

[2254 rows x 1 columns]


In [2]:
# Load data from Excel
sheet2_df = pd.read_csv("MKC_data.csv")
#sheet2_df = pd.read_excel('Update.xlsx', sheet_name='Sheet2')

# Remove newline characters from all columns
sheet2_df = sheet2_df.replace('\n', '', regex=True)
sheet2_df.columns = ['SKU', 'Color', 'Size', 'Stok']

#print(sheet2_df)

      SKU        Color         Size  Stok
0    7503         Biru     All size     0
1    7503       Kuning     All size     0
2    7503        Putih     All size     0
3    7503        Hitam     All size     0
4    7503  Orange bata     All size     0
..    ...          ...          ...   ...
811  H431   Navy batik     Big size     0
812  H431   Navy batik  Super jumbo     0
813  H431  Cream batik     All size    10
814  H431  Cream batik     Big size    10
815  H431  Cream batik  Super jumbo     9

[816 rows x 4 columns]


In [3]:
# Extracting SKU, Color, and Size from Reference_Code in Sheet1
split_cols = sheet1_df['Reference_Code'].str.split('-', expand=True)
if len(split_cols.columns) == 4:
    split_cols.columns = ['SKU', 'Color', 'Size', 'Stok']

# Rename columns in sheet1_df
# split_cols.columns = ['Production_Code', 'SKU', 'Color', 'Size']
#split_cols = split_cols.drop(['unknown_1'], axis=1)

#print(split_cols.head(20))

     SKU       Color        Size  Stok
0    NaN         NaN         NaN   NaN
1    NaN         NaN         NaN   NaN
2    NaN         NaN         NaN   NaN
3   H431  Hitambatik     Allsize  None
4   H431  Hitambatik     Bigsize  None
5   H431      Maroon     Bigsize  None
6   H431      Maroon     Allsize  None
7   H431  Merahbatik     Bigsize  None
8   H431       Hitam  Superjumbo  None
9   H431  Creambatik     Bigsize  None
10  H431   Navybatik     Allsize  None
11  H431  Merahbatik  Superjumbo  None
12  H431   Navybatik  Superjumbo  None
13  H431      Maroon  Superjumbo  None
14  H431       Hijau     Allsize  None
15  H431       Hijau     Bigsize  None
16  H431  Creambatik  Superjumbo  None
17  H431  Merahbatik     Allsize  None
18  H431  Creambatik     Allsize  None
19  H431   Navybatik     Bigsize  None


In [4]:
# Concatenating temp_df with sheet1_df
sheet1_df = split_cols

# Cleaning Color column in sheet2
sheet1_df['Color'] = sheet1_df['Color'].str.lower().str.replace(' ', '')

# Cleaning Size column in sheet2
sheet1_df['Size'] = sheet1_df['Size'].str.lower().str.replace(' ', '')

#print(sheet1_df.iloc[222])

SKU                 8863
Color    motifkotakmerah
Size             allsize
Stok                None
Name: 222, dtype: object


In [5]:
sheet1_df['SKU'] = sheet1_df['SKU'].astype(str)
sheet2_df['SKU'] = sheet2_df['SKU'].astype(str)

# Cleaning Color column in sheet2
sheet2_df['Color'] = sheet2_df['Color'].str.lower().str.replace(' ', '')
# Cleaning Size column in sheet2
sheet2_df['Size'] = sheet2_df['Size'].str.lower().str.replace(' ', '')


In [6]:
# Merge Sheet1 and Sheet2 on SKU, Color, and Size
merge_cols = ['SKU', 'Color', 'Size']
merged_df = pd.merge(sheet1_df, sheet2_df[['SKU', 'Color', 'Size', 'Stok']], on=merge_cols, how='left', suffixes=('', '_sheet2'))

# Update Stok in Sheet1 with Stok from Sheet2 where available
merged_df['Stok'] = merged_df['Stok_sheet2'].fillna(merged_df['Stok'])
merged_df = merged_df.drop('Stok', axis=1)

merged_df.columns = ['SKU', 'Color', 'Size', 'Stok']

merged_df['Stok'] = merged_df['Stok'].fillna(0)  # Fill NaN values with zero for easier

#print(merged_df.head(5))


    SKU       Color     Size  Stok
0   nan         NaN      NaN   0.0
1   nan         NaN      NaN   0.0
2   nan         NaN      NaN   0.0
3  H431  hitambatik  allsize   0.0
4  H431  hitambatik  bigsize   5.0


In [7]:
# Save the updated DataFrame back to Excel
merged_df.to_excel('Copy to Mass Update.xlsx', index=False)